# Set-up

In [ ]:
import digitalhub as dh

project = dh.get_or_create_project("demo-ml")

# Generate data

In [ ]:
%%writefile data-prep.py

import pandas as pd
from sklearn.datasets import load_breast_cancer

from digitalhub_runtime_python import handler

@handler(outputs=["dataset"])
def breast_cancer_generator():
    """
    A function which generates the breast cancer dataset
    """
    breast_cancer = load_breast_cancer()
    breast_cancer_dataset = pd.DataFrame(
        data=breast_cancer.data, columns=breast_cancer.feature_names
    )
    breast_cancer_labels = pd.DataFrame(data=breast_cancer.target, columns=["target"])
    breast_cancer_dataset = pd.concat(
        [breast_cancer_dataset, breast_cancer_labels], axis=1
    )

    return breast_cancer_dataset

In [ ]:
data_gen_fn = project.new_function(name="data-prep",
                                   kind="python",
                                   python_version="PYTHON3_9",
                                   code_src="data-prep.py",
                                   handler="breast_cancer_generator")

In [ ]:
gen_data_run = data_gen_fn.run(action="job", outputs={"dataset": "dataset"}, local_execution=True)

In [ ]:
gen_data_run.output("dataset").as_df().head()

# Train the model

In [ ]:
%%writefile train-model.py


import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from digitalhub_runtime_python import handler
from sklearn.svm import SVC
from pickle import dump
import sklearn.metrics
import os

@handler(outputs=["dataset"])
def train(project, di):

    df_cancer = di.as_df()
    X = df_cancer.drop(['target'],axis=1)
    y = df_cancer['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=5)
    svc_model = SVC()
    svc_model.fit(X_train, y_train)
    y_predict = svc_model.predict(X_test)

    if not os.path.exists("model"):
        os.makedirs("model")

    with open("model/cancer_classifier.pkl", "wb") as f:
        dump(svc_model, f, protocol=5)

    metrics = {
        "f1_score": sklearn.metrics.f1_score(y_test, y_predict),
        "accuracy": sklearn.metrics.accuracy_score(y_test, y_predict),
        "precision": sklearn.metrics.precision_score(y_test, y_predict),
        "recall": sklearn.metrics.recall_score(y_test, y_predict),
    }
    project.log_model(
            name="cancer_classifier",
            kind="sklearn",
            source="./model/",
            metrics=metrics
    )

In [ ]:
train_fn = project.new_function(name="train",
                                kind="python",
                                python_version="PYTHON3_10",
                                code_src="train-model.py",
                                handler="train",
                                requirements=["scikit-learn==1.2.2"])

In [ ]:
train_run = train_fn.run(action="job", inputs={"di": gen_data_run.output("dataset").key})

In [ ]:
model = project.get_model("cancer_classifier")
model.spec.path

# Deploy the model

In [ ]:
func = project.new_function(name="serve_sklearnmodel",
                            kind="sklearnserve",
                            path=model.spec.path + 'model/cancer_classifier.pkl')

serve_run = func.run(action="serve")

In [ ]:
import numpy as np

data = np.random.rand(2, 30).tolist()
json = {
    "inputs": [
        {
        "name": "input-0",
        "shape": [2, 30],
        "datatype": "FP32",
        "data": data
        }
    ]
}

You can check the status of the model with the following. When the attribute service appears, the model is ready to be used.

In [ ]:
serve_run.refresh().status.service

In [ ]:
serve_run.invoke(json=json).json()